# Quickstart

## Basic Example

First, let's start with a basic example, to see how easy it is to switch between providers

In [2]:
from v_router import Client, LLM, BackupModel

llm_config = LLM(
        model_name="claude-sonnet-4",
        provider="anthropic",
        max_tokens=100,
        temperature=0
    )

client = Client(llm_config)

response = await client.messages.create(
    messages=[
        {"role": "user", "content": "Say hello in one sentence."}
    ]
)

print(f"Response: {response.content}")
print(f"Model: {response.model}")
print(f"Provider: {response.provider}")

Response: Hello! It's nice to meet you.
Model: claude-sonnet-4-20250514
Provider: anthropic


## Fallback Example
Here we see what happens when you set a model fallback, if the main model fails VRouter will try the fallback models 

In [3]:
llm_config = LLM(
        model_name="claude-6",  # Primary model
        provider="anthropic",
        max_tokens=100,
        backup_models=[
            BackupModel(
                model=LLM(
                    model_name="gpt-4o",
                    provider="openai"
                ),
                priority=1  # First fallback
            ),
            BackupModel(
                model=LLM(
                    model_name="gemini-1.5-pro",
                    provider="google"
                ),
                priority=2  # Second fallback
            )
        ]
    )
client = Client(llm_config)

# This will try claude-3-opus first, then gpt-4, then gemini-1.5-pro
response = await client.messages.create(
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": "What's 2+2?"}
        ]
    )
    
print(f"Response: {response.content}")
print(f"Model: {response.model}")
print(f"Provider: {response.provider}")


Primary model failed: Error code: 404 - {'type': 'error', 'error': {'type': 'not_found_error', 'message': 'model: claude-6'}}


Response: 2 + 2 equals 4.
Model: gpt-4o-2024-08-06
Provider: openai


## Cross Provider Switch
You can set try other providers = True.
If a call fails the system will try to call another provider with the same model

In [4]:
llm_config = LLM(
        model_name="claude-opus-4",
        provider="vertexai",  # Try Anthropic first
        max_tokens=100,
        try_other_providers=True  # Enable cross-provider fallback
    )

client = Client(llm_config)

response = await client.messages.create(
        messages=[
            {"role": "user", "content": "Tell me a short joke."}
        ]
    )
    
print(f"Response: {response.content}")
print(f"Model: {response.model}")
print(f"Provider: {response.provider}")

Primary model failed: Error code: 429 - {'error': {'code': 429, 'message': 'Quota exceeded for aiplatform.googleapis.com/online_prediction_input_tokens_per_minute_per_base_model with base model: anthropic-claude-opus-4. Please submit a quota increase request. https://cloud.google.com/vertex-ai/docs/generative-ai/quotas-genai.', 'status': 'RESOURCE_EXHAUSTED'}}


Response: Why don't scientists trust atoms?

Because they make up everything!
Model: claude-opus-4-20250514
Provider: anthropic
